In [19]:
import pandas as pd
import numpy as np
import orquest_csv_utils as ocu

utils imported


In [21]:
# Load relevant data
measures = pd.read_csv('measures_rich.csv', dtype=ocu.dftypes['measures'])
df = df_dict[current_name]
for col in df.columns:
   if col in ocu.tcasts[current_name].keys():
       match ocu.tcasts[current_name][col]:
           case 'decimal_with_comma':
               ocu.cast_comma_decimal_as_float(df,col)
           case 'dmy_hm_datetime':
               ocu.cast_d_m_y_h_m_as_datetime(df,col)
           case 'ymd_datetime':
               ocu.cast_y_m_d_as_datetime(df,col)
measures.dtypes

measure     object
date        object
value       object
store_id     int32
dtype: object

In [17]:
# Split datetime into date and time
measures['time'] = measures['date'].dt.time
measures

AttributeError: Can only use .dt accessor with datetimelike values

In [15]:
measures_split = measures.pivot_table(index=['date','store_id'],columns='measure',aggfunc='sum')

In [16]:
measures_split

value                  
measure                      FOOTFALL     SALES TICKETS
date                store_id                           
2023-11-01 10:00:00 2            33.0   277.860     8.0
2023-11-01 10:30:00 2            33.0   277.860     8.0
2023-11-01 11:00:00 2            89.0   743.090    24.0
2023-11-01 11:30:00 2            89.0   743.090    24.0
2023-11-01 12:00:00 2           161.0  1000.845    42.0
...                               ...       ...     ...
2024-01-31 20:00:00 2            60.0   233.615    19.0
2024-01-31 20:30:00 2            60.0   233.615    19.0
2024-01-31 21:00:00 2            16.0    62.370     4.0
2024-01-31 21:30:00 2            16.0    62.370     4.0
2024-01-31 22:00:00 2            16.0    62.370     4.0

[3834 rows x 3 columns]